# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
from pprint import pprint
import cartopy
import pyproj
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yuzhno-kurilsk,44.0306,145.8556,14.60,97,99,4.12,RU,1690057696
1,1,kencong,-8.2833,113.3667,20.96,85,81,1.60,ID,1690057696
2,2,udachny,66.4167,112.4000,20.25,55,12,3.99,RU,1690057696
3,3,iqaluit,63.7506,-68.5145,7.85,87,100,2.06,CA,1690057697
4,4,hamilton,39.1834,-84.5333,29.87,48,0,3.60,US,1690057474


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_data_df_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
city_data_df_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
A max temperature lower than 27 degrees but higher than 21

Wind speed less than 4.5 m/s

Zero cloudiness

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cond1 = (city_data_df["Max Temp"]>21)&(city_data_df["Max Temp"]<27)
cond2 = (city_data_df["Wind Speed"]<4.5)
cond3 = (city_data_df["Cloudiness"]==0)
city_data_df_narrow = city_data_df[cond1 & cond2 & cond3]

# Drop any rows with null values
city_data_df_narrow.dropna(inplace=True)

# Display sample data
city_data_df_narrow

C:\Users\Tim\AppData\Local\Temp\ipykernel_8716\1634972087.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df_narrow.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,jebrail,39.3992,47.0283,25.49,44,0,1.92,AZ,1690057698
37,37,buka,40.8108,69.1986,26.31,25,0,2.49,UZ,1690057705
45,45,constantia,44.1833,28.6500,26.50,69,0,2.09,RO,1690057570
93,93,g'uzor,38.6214,66.2522,25.02,20,0,2.16,UZ,1690057722
128,128,hebi,35.8992,114.1925,23.97,85,0,0.85,CN,1690057729
223,223,fort mcmurray,56.7268,-111.3810,26.77,53,0,3.09,CA,1690057623
283,283,slave lake,55.2834,-114.7690,24.99,61,0,3.09,CA,1690057764
325,325,sant lluis,39.8526,4.2594,26.22,65,0,3.60,ES,1690057776
334,334,hauterive,46.0901,3.4459,23.42,56,0,0.43,FR,1690057779
357,357,lazarevskoye,43.9061,39.3331,23.34,81,0,1.14,RU,1690057784


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrow.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,jebrail,AZ,39.3992,47.0283,44,
37,buka,UZ,40.8108,69.1986,25,
45,constantia,RO,44.1833,28.6500,69,
93,g'uzor,UZ,38.6214,66.2522,20,
128,hebi,CN,35.8992,114.1925,85,
223,fort mcmurray,CA,56.7268,-111.3810,53,
283,slave lake,CA,55.2834,-114.7690,61,
325,sant lluis,ES,39.8526,4.2594,65,
334,hauterive,FR,46.0901,3.4459,56,
357,lazarevskoye,RU,43.9061,39.3331,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    
   "apiKey": geoapify_key,
    "radius":radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Long =row["Lng"]
    Lat =row["Lat"]
    City =row["City"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"circle:{Long},{Lat},{radius}"
    #params["bias"] = f"proximity:{Long},{Lat}"
    #params["city"] = f"city:{City}"

    
    params = {
         "categories" : "accommodation.hotel",
         #"text": my_address,
         "apiKey": geoapify_key,
         "filter": "circle:47.0283,39.3992,10000",
         "bias" : "proximity:47.0283,39.3992",
         "radius" : 10000,
         "city" : "city:" + City,
         #"country" : "france"
        }

    # Print the json (pretty printed)

    
    
    #print('FILTER', f"circle:{Long},{Lat},{radius}")
    #print('BIAS',f"proximity:{Long},{Lat}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    features = name_address['features']
    print("CITY", City, features)
    continue
    
    # Convert the API response to JSON format
    name_address = json.dumps(name_address)
            

    print(json.dumps(name_address))
    print('___________________________')
    #print(name_address["features"]) #[0]) # ["properties"]["name"])
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    break

# Display sample data
hotel_df

Starting hotel search
CITY jebrail []
CITY buka []
CITY constantia []
CITY g'uzor []
CITY hebi []
CITY fort mcmurray []
CITY slave lake []
CITY sant lluis []
CITY hauterive []
CITY lazarevskoye []
CITY jurm []
CITY colonia []


,City,Country,Lat,Lng,Humidity,Hotel Name
7,jebrail,AZ,39.3992,47.0283,44,
37,buka,UZ,40.8108,69.1986,25,
45,constantia,RO,44.1833,28.6500,69,
93,g'uzor,UZ,38.6214,66.2522,20,
128,hebi,CN,35.8992,114.1925,85,
223,fort mcmurray,CA,56.7268,-111.3810,53,
283,slave lake,CA,55.2834,-114.7690,61,
325,sant lluis,ES,39.8526,4.2594,65,
334,hauterive,FR,46.0901,3.4459,56,
357,lazarevskoye,RU,43.9061,39.3331,81,


In [7]:
import json
my_address = "1309 Hay St, West Perth WA 6005, Australia"

params = {
    #"text": my_address,
    "apiKey": geoapify_key,
    #"filter": "circle:47.0283,39.3992,10000",
    #"bias" : "proximity:47.0283,39.3992",
    "radius" : 10000,
    "city" : "city:paris",
    "country" : "france"
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v1/geocode/search"
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "bbox": [
                2.3508132,
                48.8557996,
                2.3523161,
                48.8574578
            ],
            "geometry": {
                "coordinates": [
                    2.351437689955218,
                    48.85659475
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "City Hall Plaza",
                "address_line2": "75004 Paris, France",
                "city": "Paris",
                "city_block": "Quartier Saint-Merri",
                "country": "France",
                "country_code": "fr",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database License",
                    "sourcename": "openstreetmap",
                    "url": "https://www.openstreetmap.org/copyright"
                },
                "district": "Pa

In [22]:
import json
my_address = "1309 Hay St, West Perth WA 6005, Australia"

params = {
    "categories" : 'accommodation.hotel',
    "filter" : 'circle:-111.38103,56.72676,10000',  # lon, lat, radius
    "limit" : 5,
    "apiKey" : geoapify_key
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v2/places"
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -111.38011500382473,
                    56.7230757184376
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Clearwater Suite Hotel",
                "address_line2": "4 Haineault Street, Fort McMurray, AB T9H 1L2, Canada",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "building.accommodation"
                ],
                "city": "Fort McMurray",
                "country": "Canada",
                "country_code": "ca",
                "county": "Division No. 16",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:ci

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [ ]:
for x in data_list:
    #x = x['features'] #[0]['properties']
    pprint(x)
    print('---------------')